In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
electorate_shapes = gpd.read_file("raw/CED_2024_AUST_GDA2020/CED_2024_AUST_GDA2020.shp")[["CED_CODE24", "CED_NAME24", "geometry"]].rename({"CED_NAME24":"2025Division","CED_CODE24":"2025DivisionCode"},axis=1)
polling_places = pd.read_csv("raw/GeneralPollingPlacesDownload-27966.csv",header=1,index_col="PollingPlaceID")[["PollingPlaceNm","Latitude", "Longitude","DivisionNm"]].rename({"DivisionNm":"2022Division"},axis=1)


In [4]:
gdf_points = gpd.GeoDataFrame(polling_places[["PollingPlaceNm","2022Division"]], geometry=gpd.points_from_xy(polling_places.Longitude, polling_places.Latitude), crs="EPSG:7844")


In [5]:
gdf_joined = gpd.sjoin(gdf_points, electorate_shapes, how="left", predicate="within")


In [6]:
PPMAPPER = gdf_joined.dropna()['2025Division']

In [7]:
NSW_votes2022 = pd.read_csv('raw/primaries/HouseStateFirstPrefsByPollingPlaceDownload-27966-NSW.csv',header=1)
NSW_votes2022allVoteTypes = pd.read_csv("raw/primaries/HouseFirstPrefsByCandidateByVoteTypeDownload-27966.csv",header=1)
NSW_votes2022allVoteTypes = NSW_votes2022allVoteTypes[NSW_votes2022allVoteTypes["StateAb"]=="NSW"][["DivisionNm", "PartyAb", "Surname", "GivenNm","OrdinaryVotes", "AbsentVotes", "ProvisionalVotes", "PrePollVotes", "PostalVotes", "TotalVotes"]]


In [8]:
def handle_row(row):
    if row['PollingPlaceID'] in PPMAPPER.index:
        newDIV = PPMAPPER.loc[row['PollingPlaceID']]
    else:newDIV = row['DivisionNm']

    if pd.isna(row['PartyAb']) and row['Surname']=="Informal":
        partyab = "INF"
    elif pd.isna(row['PartyAb']):
        partyab = "IND"
    else:
        partyab = row['PartyAb']

    og_numPPIDs = len(NSW_votes2022[NSW_votes2022["DivisionNm"]==row['DivisionNm']]["PollingPlaceID"].unique())
    candidate_idx = NSW_votes2022allVoteTypes[(NSW_votes2022allVoteTypes[["DivisionNm","Surname", "GivenNm"]] == [row['DivisionNm'],row['Surname'], row['GivenNm']]).all(axis=1)].index
    if len(candidate_idx)==1:
        candidate_idx=candidate_idx[0]
    else:
        print([row['DivisionNm'],row['Surname'], row['GivenNm'],row['PartyAb']])
        print(candidate_idx)
        raise Exception("Error multiple candidates detected. Should only be one.")

    candidate_absentee = NSW_votes2022allVoteTypes.loc[candidate_idx]["AbsentVotes"]
    candidate_provisional = NSW_votes2022allVoteTypes.loc[candidate_idx]["ProvisionalVotes"]
    candidate_prepoll = NSW_votes2022allVoteTypes.loc[candidate_idx]["PrePollVotes"]
    candidate_postal = NSW_votes2022allVoteTypes.loc[candidate_idx]["PostalVotes"]



    return pd.Series(
        {"PollingPlaceID":row['PollingPlaceID'],
         "DivisionNm":newDIV,
         "PartyAb":partyab,
         "Surname":row['Surname'],
         "GivenNm":row["GivenNm"],
         "OrdinaryVotes":row["OrdinaryVotes"],
         "AbsentVotes":candidate_absentee/og_numPPIDs,
         "ProvisionalVotes":candidate_provisional/og_numPPIDs, 
         "PrePollVotes":candidate_prepoll/og_numPPIDs,
         "PostalVotes": candidate_postal/og_numPPIDs,
         "TotalVotes": row["OrdinaryVotes"] + (candidate_absentee + candidate_provisional + candidate_prepoll + candidate_postal)/og_numPPIDs
         }
    )
        
NSW_votes2025 = NSW_votes2022.apply(handle_row,axis=1)

In [9]:
votes = NSW_votes2025.pivot_table(values="TotalVotes", index="DivisionNm", columns="PartyAb", aggfunc="sum")

In [10]:
votes.drop("INF",axis=1)

PartyAb,AJP,ALP,ASP,AUD,CEC,CYA,DPDA,GRN,IAP,IMO,...,LDP,LP,NAFD,NP,ON,SAL,SOPA,SPP,TNL,UAPP
DivisionNm,,,,,,,,,,,,,,,,,,,,,
Banks,NaN,27064.957701,NaN,NaN,NaN,NaN,NaN,6835.387785,NaN,NaN,...,975.192308,35993.888811,732.673077,NaN,2663.503437,NaN,NaN,NaN,NaN,4590.067368
Barton,NaN,47219.574596,NaN,NaN,NaN,NaN,NaN,10611.260022,NaN,NaN,...,228.192308,32163.393268,181.173077,NaN,4655.633510,NaN,NaN,NaN,NaN,6447.468408
Bean,NaN,308.103448,NaN,2.896552,NaN,NaN,NaN,81.333333,NaN,7.310345,...,26.620690,198.344828,NaN,NaN,37.206897,NaN,NaN,17.827586,NaN,15.942529
Bennelong,103.027721,35886.327781,NaN,NaN,NaN,NaN,790.942308,11459.995660,NaN,183.381818,...,1538.638667,45340.500430,NaN,NaN,1759.286671,NaN,1431.961538,411.672727,304.110985,3051.993147
Berowra,41.827586,30654.617344,NaN,NaN,NaN,466.272727,258.807692,17867.142273,NaN,NaN,...,2410.297815,57316.833686,NaN,NaN,3337.278425,NaN,1593.447552,NaN,NaN,3149.026082
Blaxland,391.103448,37071.050231,NaN,NaN,NaN,NaN,NaN,4871.444022,NaN,NaN,...,735.497731,19120.080734,NaN,NaN,3145.147056,NaN,56.692308,NaN,NaN,5104.506922
Bradfield,7.372549,18326.038878,NaN,NaN,NaN,NaN,NaN,8720.105507,NaN,165.018182,...,358.490909,46166.998593,NaN,NaN,1509.483329,NaN,NaN,389.127273,322.018182,2356.408387
Calare,NaN,15979.655172,NaN,NaN,NaN,NaN,NaN,4819.551724,NaN,NaN,...,NaN,NaN,NaN,50407.448276,8934.620690,NaN,NaN,NaN,NaN,3983.793103
Canberra,213.806228,16099.358003,127.922042,13.793103,23.078357,29.198299,13.269231,5512.255846,82.553531,127.674622,...,999.316497,14024.803065,10.038462,2274.983188,2094.381452,23.762712,115.785437,69.673354,23.975881,1635.880352


In [11]:
votes.sum().sum()

4959584.0

In [12]:
NSW_votes2022allVoteTypes["TotalVotes"].sum()

4959584